In [1]:
import cuml
import cudf
import os
import numpy as np
import cupy as cp
import dask_ml.model_selection as dms
from cuml.model_selection import train_test_split
from cuml.linear_model import MBSGDClassifier,LogisticRegression
from cuml.svm import SVC
from cuml.ensemble import RandomForestClassifier
from cuml.preprocessing import LabelEncoder
from cuml.metrics import confusion_matrix



In [2]:
#reading csv using cudf which gpu alternative for pandas
cdf= cudf.read_csv('heart.csv')
encoder=LabelEncoder()

In [4]:
# encoding any columns that's not in numerical
for i in cdf.columns:
    if(cdf[i].dtype=='object'):
        cdf[i]=encoder.fit_transform(cdf[i])

In [5]:
#creating a x and y to seperate independent and dependent variables
x=cdf.drop(['HeartDisease'],axis=1)
y=cdf['HeartDisease']
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.25)

In [14]:
#initializing multiple ml algorithms
lr=LogisticRegression()
sgd=MBSGDClassifier()
svc=SVC()
rfc=RandomForestClassifier()

In [7]:
#function to train the models and display confusion matrix
def results(model,cdf):
    x=cdf.drop(['HeartDisease'],axis=1)
    y=cdf['HeartDisease']
    x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.25,random_state=42)
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    print(confusion_matrix(y_test,y_pred))

In [8]:
results(lr,cdf)

[[ 82  14]
 [ 17 116]]


In [9]:
results(sgd,cdf)

[[ 78  18]
 [ 19 114]]


In [10]:
results(svc,cdf)

[[70 26]
 [44 89]]


In [11]:
results(rfc,cdf)

[[ 83  13]
 [ 19 114]]


/packages/7x/anaconda3/5.3.0/envs/rapids-22.02-cuda-11.5/lib/python3.9/site-packages/cuml/internals/api_decorators.py:567: UserWarning: To use pickling or GPU-based prediction first train using float32 data to fit the estimator
  ret_val = func(*args, **kwargs)
/packages/7x/anaconda3/5.3.0/envs/rapids-22.02-cuda-11.5/lib/python3.9/contextlib.py:79: UserWarning: GPU based predict only accepts np.float32 data. The model was trained on np.float64 data hence cannot use GPU-based prediction! 
Defaulting to CPU-based Prediction. 
To predict on float-64 data, set parameter predict_model = 'CPU'
  return func(*args, **kwds)


The RandomForestClassifier takes input in float32 for GPU computation and auntomatically converts it into float32 during inference if it doesn't we need to convert the training data to float32 but confusion matrix works on int so we converted the predicted result to int64

In [33]:
rfc.fit(x_train.astype(cp.float32),y_train.astype(cp.float32))
confusion_matrix(y_test,rfc.predict(x_test).astype(cp.int64))

array([[ 88,  14],
       [ 14, 113]])

In [16]:
rfc.fit(x_train,y_train)
y_pred=rfc.predict(x_test,predict_model='CPU')

We can conclude that the package rapidai is imple to use and helps us in leveraging the gpu for both data procesing and ml implementation 